In [ ]:
# import sys
# !{sys.executable} -m pip install imbalanced-learn --user
# !{sys.executable} -m pip install hyperopt --user

In [25]:
from time import time
from sklearn.model_selection import GridSearchCV
from hyperopt import tpe, fmin, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from copy import deepcopy

from newsgac import config
from newsgac.genres import genre_codes
from newsgac.learners import learners, LearnerSVC, LearnerNB, LearnerXGB, LearnerGB, LearnerMLP, LearnerRF, LearnerLGBM
from newsgac.pipelines.get_sk_pipeline import get_sk_pipeline
from newsgac.pipelines.utils import report
from sklearn.neural_network import MLPClassifier

from newsgac import database
from newsgac.data_sources import DataSource
from newsgac.pipelines import Pipeline

import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/newsgac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
[d.display_title for d in DataSource.objects.all()]

[u'N2BGS Training',
 u'N2BGS Testing',
 u'Linked NRC (2930)',
 u'BOW + 5 features (N2BGS train)',
 u'BOW + 5 features (N2BGS test)',
 u'nrc-1950-1994-a.txt',
 u'nrc-1950-1994-b.txt',
 u'nrc-1950-1994-c.txt',
 u'nrc-1950-1994-d.txt',
 u'nrc-1950-1994-e.txt',
 u'telegraaf-1950-1994-a.txt',
 u'telegraaf-1950-1994-b.txt',
 u'telegraaf-1950-1994-c.txt',
 u'telegraaf-1950-1994-d.txt',
 u'telegraaf-1950-1994-e.txt',
 u'volkskrant-1950-1995-a.txt',
 u'volkskrant-1950-1995-b.txt',
 u'volkskrant-1950-1995-c.txt',
 u'volkskrant-1950-1995-d.txt',
 u'volkskrant-1950-1995-e.txt',
 u'nrc-1965.txt',
 u'nrc-1985.txt',
 u'BOW + 9 features (N2BGS train)',
 u'BOW + 9 features (N2BGS test)',
 u'Linked NRC (2930/9 features)',
 u'BOW Train unique (9 features)',
 u'BOW Test unique (9 features)',
 u'Linked NRC (unique/9 features)',
 u'BOW Train unique (N3BGS/9 features)',
 u'N3BGS FROG Test',
 u'N3BGS FROG Train',
 u'BOW Test unique (N3BGS/9 features)',
 u'BOW Test unique (N3BGS/9 features/collapsed)',
 u'Link

In [27]:
[d.display_title for d in Pipeline.objects.all()]

[u'dummy frog',
 u'dummy bow',
 u'RF FROG',
 u'SVC FROG',
 u'XGB FROG',
 u'NB FROG',
 u'MLP FROG',
 u'RF BOW',
 u'SVC BOW',
 u'XGB BOW',
 u'NB BOW',
 u'MLP BOW',
 u'SVC LIN BOW',
 u'Erik MLP BOW (with stop-words)',
 u'Erik MLP BOW (5 features) ',
 u'Erik MLP BOW (9 features) ',
 u'Erik SVC BOW (9 features)',
 u'Erik RF BOW (9 features)',
 u'Erik XGB BOW (9 features)',
 u'Erik NB BOW (9 features)',
 u'Erik MLP unique 9 features',
 u'Erik NB unique (9 features)',
 u'Erik XGB unique (9 features)',
 u'Erik SVC unique (9 features)',
 u'Erik RF unique (9 features)',
 u'N3BGS MLP unique 9 features',
 u'N3BGS NB unique 9 features ',
 u'N3BGS SVC unique 9 features ',
 u'N3BGS RF unique 9 features',
 u'N3BGS XGB unique 9 features ',
 u'dummy n3bgs bow',
 u'dummy n3bgs frog',
 u'RF FROG N3BGS',
 u'SVC FROG N3BGS',
 u'XGB FROG N3BGS',
 u'Erik NB N3BGS Collapsed',
 u'Erik MLP N3BGS Collapsed',
 u'Erik MLP Linked NRC (2930 articles)',
 u'Erik RF Linked NRC (2930 articles) ',
 u'Erik NB Linked NRC (2

In [29]:
p = Pipeline.objects.all()[65]
print 'DataSource: ' + p.data_source.display_title
print 'NLP Tool: ' + p.nlp_tool.name
print 'Classifier: ' + p.learner.name
print 'Task status: ' + str(p.task.status)

DataSource: 20190228 unbalanced train size=3443
NLP Tool: TF-IDF
Classifier: Light GBM
Task status: Status.SUCCESS


In [30]:
len(p.data_source.articles)

3099

In [31]:
test_data_source = DataSource.objects.all()[39]
print 'Testing data source: ' + test_data_source.display_title

Testing data source: 20190228 unbalanced test size=3443


In [32]:
skp = p.get_sk_pipeline()
skp_opt = deepcopy(skp)
skp_opt.steps.pop()
skp_opt.steps

[('CleanOCR', <newsgac.nlp_tools.transformers.CleanOCR at 0x7fda5a3ad390>),
 ('StopWordRemoval',
  <newsgac.nlp_tools.transformers.StopWordRemoval at 0x7fda5a3ad590>),
 ('FeatureExtraction', FeatureUnion(n_jobs=None,
         transformer_list=[('TFIDF', Pipeline(memory=None,
       steps=[('RemoveQuotes', <newsgac.nlp_tools.transformers.RemoveQuotes object at 0x7fda5a964510>), ('TF-IDF', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
          dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
          lowercase=True, max_df=1.0, max_featu...        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('RobustScaler',
  RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=False,
         with_scaling=True))]

In [33]:
texts = np.array([article.raw_text for article in p.data_source.articles])
labels = np.array([article.label for article in p.data_source.articles])

X = skp_opt.fit_transform(texts)

In [34]:
# get the test dataset for optimization accuracy
texts_test = np.array([article.raw_text for article in test_data_source.articles])
labels_test = np.array([article.label for article in test_data_source.articles])

X_test = skp_opt.transform(texts_test)

In [35]:
X_train = X
y_train = labels
y_test = labels_test

In [36]:
y_test.shape

(344,)

In [37]:
# hidden_layer_sizes_list =[(20, 30), (30, 10), (10, 20), (5,10)]
hidden_layer_sizes_list =[(200, 100), (100, 200), (150, 300), (300, 100), (100, 200, 100), (200,100,200)]
activation_list = ['identity', 'logistic', 'tanh', 'relu']
solver_list = ['lbfgs','sgd', 'adam']
learning_rate_list = ['constant','adaptive', 'invscaling']
batch_size_list = np.arange(20, 200, dtype=int)
max_iter_list = np.arange(20, 500, dtype=int)

In [38]:
def optimize(evals, trials, optimizer=tpe.suggest):
    space = {
        'hidden_layer_sizes': hp.choice('hidden_layer_sizes', hidden_layer_sizes_list),
        'activation': hp.choice('activation', activation_list),
        'solver': hp.choice('solver', solver_list),
        'alpha': hp.lognormal('alpha', 0, 1), # [0.0001, 0.05],
#         'alpha': hp.loguniform('alpha', np.log(0.1), np.log(10)),
        'learning_rate': hp.choice('learning_rate', learning_rate_list),
        'batch_size': hp.choice('batch_size', batch_size_list),
        'max_iter': hp.choice('max_iter', max_iter_list),
    }
    best = fmin(score, space, algo=optimizer, max_evals=evals, trials=trials)
    pbar.close()
    return best

In [39]:
train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []

def score(params):
     
    model = MLPClassifier(
            hidden_layer_sizes = params['hidden_layer_sizes'],
            activation = params['activation'],
            solver = params['solver'],
            alpha = params['alpha'],
            learning_rate = params['learning_rate'],
            batch_size = params['batch_size'],
            max_iter = params['max_iter'],
            shuffle = True,
            random_state = 42,
            warm_start = False,
            early_stopping = True,
            validation_fraction = 0.1,
    )

    model.fit(X_train, y_train)   

    train_acc = accuracy_score(y_train, model.predict(X_train))
    train_loss = log_loss(y_train, model.predict_proba(X_train))
    test_acc = accuracy_score(y_test, model.predict(X_test))
    test_loss = log_loss(y_test, model.predict_proba(X_test))
    
    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
    test_acc_list.append(test_acc)
    test_loss_list.append(test_loss)
    
    print('Parameters with this training accuracy {} and loss {} :'.format(train_acc, train_loss))
    print('Parameters with this testing accuracy {} and loss {} :'.format(test_acc, test_loss))
    print(params)
    pbar.update()
#     return {'loss': test_loss, 'status': STATUS_OK}
    return {'loss': -test_acc, 'status': STATUS_OK}

In [ ]:
import time
from tqdm import tqdm

trials = Trials()
cores = 48
start = time.time()
evaluations = 1000
pbar = tqdm(total=evaluations, desc="Hyperopt")
best_param = optimize(evals=evaluations,
                      optimizer=tpe.suggest,
                      trials=trials)
print("------------------------------------")
print("The best hyperparameters are: ", "\n")
print(best_param)
end = time.time()
print('Time elapsed to optimize {0} executions: {1}'.format(evaluations, end - start))
best_param['hidden_layer_sizes'] = hidden_layer_sizes_list[best_param['hidden_layer_sizes']]
best_param['activation'] = activation_list[best_param['activation']]
best_param['solver'] = solver_list[best_param['solver']]
best_param['learning_rate'] = learning_rate_list[best_param['learning_rate']]
best_param['batch_size'] = batch_size_list[best_param['batch_size']]
best_param['max_iter'] = max_iter_list[best_param['max_iter']]
print('\n Best score:')
score(best_param)

Hyperopt:   0%|          | 0/1000 [00:00<?, ?it/s]


  0%|          | 0/1000 [00:00<?, ?it/s, best loss: ?]
Parameters with this training accuracy 0.429493384963 and loss 1.95003853758 :

  0%|          | 0/1000 [00:51<?, ?it/s, best loss: ?]
Parameters with this testing accuracy 0.468023255814 and loss 1.90519702132 :

  0%|          | 0/1000 [00:51<?, ?it/s, best loss: ?]
{'solver': 'sgd', 'hidden_layer_sizes': (200, 100, 200), 'alpha': 1.1171298533861638, 'activation': 'identity', 'max_iter': 474, 'learning_rate': 'constant', 'batch_size': 167}

  0%|          | 0/1000 [00:51<?, ?it/s, best loss: ?]

Hyperopt:   0%|          | 1/1000 [00:52<14:28:47, 52.18s/it]


  0%|          | 1/1000 [00:52<14:25:57, 52.01s/it, best loss: -0.468023255814]
Parameters with this training accuracy 0.780251694095 and loss 0.634446304926 :

  0%|          | 1/1000 [03:16<14:25:57, 52.01s/it, best loss: -0.468023255814]
Parameters with this testing accuracy 0.508720930233 and loss 2.18926604528 :

  0%|          | 1/1000 [03:16<14:25:57, 52.01s/it, best loss: -0.468023255814]
{'solver': 'lbfgs', 'hidden_layer_sizes': (100, 200, 100), 'alpha': 0.7641217625681462, 'activation': 'logistic', 'max_iter': 290, 'learning_rate': 'constant', 'batch_size': 92}

  0%|          | 1/1000 [03:16<14:25:57, 52.01s/it, best loss: -0.468023255814]

Hyperopt:   0%|          | 2/1000 [03:16<22:09:49, 79.95s/it]

...


  1%|          | 7/1000 [11:24<31:32:11, 114.33s/it, best loss: -0.572674418605]
Parameters with this training accuracy 0.429493384963 and loss 1.95143311843 :

  1%|          | 7/1000 [14:52<31:32:11, 114.33s/it, best loss: -0.572674418605]
Parameters with this testing accuracy 0.470930232558 and loss 1.90632541999 :

  1%|          | 7/1000 [14:52<31:32:11, 114.33s/it, best loss: -0.572674418605]
{'solver': 'sgd', 'hidden_layer_sizes': (200, 100, 200), 'alpha': 1.184029022136145, 'activation': 'identity', 'max_iter': 418, 'learning_rate': 'adaptive', 'batch_size': 149}

  1%|          | 7/1000 [14:52<31:32:11, 114.33s/it, best loss: -0.572674418605]

Hyperopt:   1%|          | 8/1000 [14:52<39:17:25, 142.59s/it]


  1%|          | 8/1000 [14:52<39:17:10, 142.57s/it, best loss: -0.572674418605]
Parameters with this training accuracy 0.431106808648 and loss 1.99824680332 :

  1%|          | 8/1000 [27:27<39:17:10, 142.57s/it, best loss: -0.572674418605]
Parameters with this testing accuracy 0.468023255814 and loss 1.9581148206 :

  1%|          | 8/1000 [27:27<39:17:10, 142.57s/it, best loss: -0.572674418605]
{'solver': 'sgd', 'hidden_layer_sizes': (200, 100, 200), 'alpha': 0.5395456678062278, 'activation': 'logistic', 'max_iter': 186, 'learning_rate': 'adaptive', 'batch_size': 32}

  1%|          | 8/1000 [27:27<39:17:10, 142.57s/it, best loss: -0.572674418605]

Hyperopt:   1%|          | 9/1000 [27:27<89:48:42, 326.26s/it]


  1%|          | 9/1000 [27:27<89:48:32, 326.25s/it, best loss: -0.572674418605]

In [ ]:
# get the best params and train the model
mlp_opt = MLPClassifier(
            hidden_layer_sizes = best_param['hidden_layer_sizes'],
            activation = best_param['activation'],
            solver = best_param['solver'],
            alpha = best_param['alpha'],
            learning_rate = best_param['learning_rate'],
            batch_size = best_param['batch_size'],
            max_iter = best_param['max_iter'],
            shuffle = True,
            random_state = 42,
            warm_start = False,
            early_stopping = True,
            validation_fraction = 0.1,
    )

fitted_model = mlp_opt.fit(X=X_train,y=y_train)

In [ ]:
y_score = fitted_model.predict_proba(X_test)
y_score

In [ ]:
test_acc = accuracy_score(y_test, fitted_model.predict(X_test))
test_acc

In [ ]:
from matplotlib import pyplot as plt 
%matplotlib inline 
x_list =range(len(train_acc_list))

plt.plot(x_list, train_acc_list, label='Training accuracy')
plt.plot(x_list, test_acc_list, label='Testing accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_list, train_loss_list, label='Training loss')
plt.plot(x_list, test_loss_list, label='Testing loss')
plt.legend()
plt.show()